In [ ]:
# API 키 설정 (!!!본인 키 넣기!!!) (커밋 시 지우고 올리기)
API_KEY = ''

In [2]:
import requests
import pandas as pd

PLATFORM = 'steam'
START_TIME = '2025-11-01T00:00:00Z'  # 받아오고 싶은 시간 (UTC ISO8601)

URL = f'https://api.pubg.com/shards/{PLATFORM}/samples'
HEADERS = {
    'Authorization': f'Bearer {API_KEY}',
    'Accept': 'application/vnd.api+json'
}
PARAMS = {
    'filter[createdAt-start]': START_TIME
}

response = requests.get(URL, headers=HEADERS, params=PARAMS)
print('Status Code:', response.status_code)

data = response.json()

Status Code: 200


In [ ]:
# 샘플 데이터의 match id 추출
match_ids = []

if 'relationships' in data['data'] and 'matches' in data['data']['relationships']:
    matches_data = data['data']['relationships']['matches'].get('data', [])
    
    if isinstance(matches_data, list):
        for match in matches_data:
            if isinstance(match, dict) and 'id' in match:
                match_ids.append(match['id'])

print(f"Total match IDs: {len(match_ids)}")
print(f"Match IDs: {match_ids}")

Total match IDs: 1009
Match IDs: ['3ba23c93-5695-44e4-bdba-22b897fd02ea', '6b2fa597-562e-46a6-b430-48e7238df3ff', 'b5be840e-4026-48f6-a42c-372387765075', '63ecf02e-b76e-4435-a638-934a8f152490', '4a3b8364-1c2e-4acb-872a-ce6782233356', '28023274-143b-4199-9d50-456b5789463f', '7a1f48ce-9ad6-43ef-a3c1-c2b7c496d40c', '9ec24f22-1b91-4f2f-a5ea-c02215cca599', '6ca78aeb-d256-4473-991a-cf686c9d2f8d', 'c6820d70-e413-42ee-942b-484411fc10c9', '5d44978a-bcb2-4366-9e06-c25cbef41d9f', '95d412da-0aee-46c5-bc28-d6449d26ea00', 'db9cf864-a615-4cbe-b496-8c2cc1ec161b', 'c629db0e-b668-4fab-a3d7-21fbd2747379', 'e2505669-bf62-42a2-9dca-6e31cd6d3e64', '7931f0ad-59c7-4b09-94cd-4271e372f6be', '230b72c6-a058-4aac-bf3e-d394b673b8f4', 'f5f48cb5-27fa-47a1-8a87-5fbf9dcacbc8', '1b554305-3969-4b05-8467-f33705c33b01', '62a30465-2114-41b4-bd4f-7479beec0bca', '3d1ec094-d5c1-4144-9118-b20bcb770993', '41a091a9-1146-4241-9bba-a314daefe109', '7a15642a-45c2-4168-a215-a749698b64e3', '88803e1c-e33d-45a7-800f-308022793e14', 'fad33

In [14]:
# 각 match id로 상세 정보 요청 (간소화)
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Accept': 'application/vnd.api+json'
}

match_data_list = []
for match_id in match_ids:
    url = f'https://api.pubg.com/shards/{PLATFORM}/matches/{match_id}'
    response = requests.get(url, headers=headers)
    
    if response.ok:
        data = response.json()
        attrs = data['data']['attributes']
        
        # telemetry URL 찾기
        telemetry_url = next(
            (item['attributes']['URL'] for item in data.get('included', []) 
             if item.get('attributes', {}).get('URL')),
            None
        )
        
        # 매치 정보 저장
        match_data_list.append({
            'id': data['data']['id'],
            'createdAt': attrs.get('createdAt'),
            'gameMode': attrs.get('gameMode'),
            'mapName': attrs.get('mapName'),
            'duration': attrs.get('duration'),
            'telemetryURL': telemetry_url,
            'isCustomGame': attrs.get('isCustomMatch'),
            'shardId': attrs.get('shardId'),
        })
    else:
        print(f"Failed to get match {match_id}: {response.status_code}")

# DataFrame으로 변환
df = pd.DataFrame(match_data_list)
print(f"Total records: {len(df)}")
print(df.head())

Total records: 1009
                                     id             createdAt gameMode  \
0  3ba23c93-5695-44e4-bdba-22b897fd02ea  2025-10-31T23:37:02Z    squad   
1  6b2fa597-562e-46a6-b430-48e7238df3ff  2025-10-31T23:55:41Z     solo   
2  b5be840e-4026-48f6-a42c-372387765075  2025-11-01T00:05:13Z     solo   
3  63ecf02e-b76e-4435-a638-934a8f152490  2025-10-31T23:42:14Z    squad   
4  4a3b8364-1c2e-4acb-872a-ce6782233356  2025-10-31T23:51:02Z     solo   

         mapName  duration                                       telemetryURL  \
0    Baltic_Main      1463  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
1  Tutorial_Main       318  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
2  Tutorial_Main       244  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
3    Desert_Main      1622  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
4     Tiger_Main      2857  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   

   isCustomGame shardId  
0         False   stea

In [15]:
# 전처리
# 맵 이름 매핑
map_name_dict = {
  "Baltic_Main": "Erangel",
  "Chimera_Main": "Paramo",
  "Desert_Main": "Miramar",
  "DihorOtok_Main": "Vikendi",
  "Erangel_Main": "Erangel",
  "Heaven_Main": "Haven",
  "Kiki_Main": "Deston",
  "Range_Main": "Camp Jackal",
  "Savage_Main": "Sanhok",
  "Summerland_Main": "Karakin",
  "Tiger_Main": "Taego",
  "Neon_Main": "Rondo"
}

# 맵 이름 딕셔너리에 있는 값 빼고 제거 & 매핑
df = df[df['mapName'].isin(map_name_dict.keys())]
df['mapName'] = df['mapName'].map(map_name_dict)

# 커스텀 게임 제거
df = df[df['isCustomGame'] == False]
df = df.drop(columns=['isCustomGame'])
df.reset_index(drop=True, inplace=True)
print(f"Records after preprocessing: {len(df)}")
print(df.head())

# 게임모드 리스트에 없는 값 제거
game_modes = ['duo', 'duo-fpp', 'squad', 'squad-fpp',
              'solo', 'solo-fpp']
df = df[df['gameMode'].isin(game_modes)]
df.reset_index(drop=True, inplace=True)

Records after preprocessing: 855
                                     id             createdAt gameMode  \
0  3ba23c93-5695-44e4-bdba-22b897fd02ea  2025-10-31T23:37:02Z    squad   
1  63ecf02e-b76e-4435-a638-934a8f152490  2025-10-31T23:42:14Z    squad   
2  4a3b8364-1c2e-4acb-872a-ce6782233356  2025-10-31T23:51:02Z     solo   
3  28023274-143b-4199-9d50-456b5789463f  2025-10-31T23:51:02Z    squad   
4  7a1f48ce-9ad6-43ef-a3c1-c2b7c496d40c  2025-11-01T00:10:37Z    squad   

   mapName  duration                                       telemetryURL  \
0  Erangel      1463  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
1  Miramar      1622  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
2    Taego      2857  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
3    Taego      1569  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   
4  Miramar      1406  https://telemetry-cdn.pubg.com/bluehole-pubg/s...   

  shardId  
0   steam  
1   steam  
2   steam  
3   steam  
4   steam  

In [ ]:
#optional: CSV로 저장
#df.to_csv('pubg_match_data.csv', index=False)

In [ ]:
#telemetryURL 이용해서 데이터 받아오기